# Gérer des données numériques dans un projet d’apprentissage supervisé

C’est bien connu, les machines adorent les nombres. Ils ont cela de commode qu’ils se prêtent mieux aux calculs que des symboles comme *pomme* ou *Pommes*. Dans un projet de *machine learning*, les données fournies aux algorithmes d’apprentissage devront toutes être représentées sous forme numérique et, avant d’aborder la manière de vectoriser des chaînes de caractère, il est plus sage de commencer par la manipulations des nombres, qu’ils prennent des valeurs entières ou réelles.

Au fil de ce calepin électronique, nous programmerons une machine intelligente censée prédire une valeur cible pour un ensemble de caractéristiques, et ce en respectant les étapes du *workflow* décrit dans la présentation :

1. Partition du jeu de données en un jeu d’entraînement et un jeu de test ;
2. entraînement du modèle avec un algorithme ;
3. évaluation de la performance du modèle.

Nous commencerons à utiliser la librairie [*Scikit-Learn*](https://scikit-learn.org/), spécifiquement dédiée aux tâches du *machine learning*, mais avant, chargeons quelques-uns des modules courants qui nous seront nécessaires pour exécuter les codes :

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_context('notebook')

## Identifier une variable aléatoire quantitative

En statistiques, une variable aléatoire est l’une des caractéristiques d’une observation. Elle peut se représenter de manière rudimentaire sous forme de tableau à deux dimensions :

|Sexe|Taille|
|-|:-:|
|F|180|
|M|172|
|M|167|
|F|178|

Le tableau étant composé de trois lignes et de deux colonnes, il est réputé présenter deux caractéristiques pour trois observations dans une structure de dimensions $4\times 2$.

Sans se tromper, la variable aléatoire *Sexe* n’est pas de type numérique quand la variable *Taille*, elle, l’est.

Comment s’en assurer avec Python ? La propriété `dtypes` d’un *data frame* répond à l’objectif :

In [ ]:
df = pd.DataFrame({
    'gender': ['F', 'M', 'M', 'F'],
    'height': [180, 172, 167, 178]
})

df.dtypes

La variable `height` est bien de type numérique. Pour autant, une variable aléatoire représentée sous forme numérique est-elle systématiquement quantitative, au sens statistique du terme ?

### Numérique ≠ quantitatif

Pour qu’une variable numérique soit considérée comme quantitative, elle est censée exprimer une quantité. Après avoir ajouté les années de naissance des individus, la propriété `dtypes` signale que la variable `birth`, conformément à l’intuition, est bien de type numérique :

In [ ]:
df["birth"] = [1983, 1987, 1987, 2001]

df.dtypes

Très bien, la variable `birth` est de type numérique (`int64`), mais est-elle quantitative ? Pour le déterminer facilement, il faut se poser la question de savoir si cela fait sens de cumuler les valeurs consignées. Quand les tailles des individus peuvent former une somme pour obtenir ensuite une moyenne, est-ce raisonnable d’additionner des années de naissance ? Le calcul de la moyenne arithmétique des années de naissance donne pour résultat : $1989,5$. Est-il logique de poser ce genre de question ?

In [ ]:
df["birth"].mean()

Eh bien, en fait, oui. Si la moyenne ici n’est pas à proprement parler intéressante, il est légitime de se demander quelle est la médiane des années de naissance des individus interrogés, elle pourrait expliquer certains résultats. En revanche, il n’en serait pas de même des codes postaux de leur lieu d’habitation ou de leurs numéros de sécurité sociale.

### Quantitative discrète ou continue ?

La différence est encore parfois plus subtile entre les variables aléatoires quantitatives discrètes et continues. Dans l’exemple avec les années, la coutume est de considérer leur valeur comme une discrétion du temps qui, lui, est continu ; d’un autre côté, si le jeu de données comporte une représentation décimale des années, où $1998,55$ équivaudrait au 15 juin 1998, rien n’interdit de considérer qu’il existe alors une continuité.

Parfois, une distinction commode véhiculée par certaines sources consiste à considérer une donnée représentée par un élément de l’ensemble $\mathbb{Z}$ comme discrète, mais appliquer cette préconisation sans réflexion peut amener à des erreurs d’interprétation. L’âge, par exemple, est souvent noté sous forme d’entiers naturels. Il s’agit d’une convention : personne ne passe réellement de $x$ ans à $x+1$ ans sans vivre les intervalles, si ? Pour s’en assurer tout à fait, demandons-nous si le calcul de l’âge moyen des individus d’une enquête ferait sens.

En revanche, si au moment de la préparation des données de l’enquête, on établissait des classes d’âge (moins de 18 ans, plus de 35 ans etc.), la variable deviendrait discrète, et qualitative. D’autres données posent les mêmes difficultés, comme la taille, ou le poids, qui, comme elles sont exprimées avec une unité et ne peuvent prendre qu’une valeur isolée, sont cataloguées généralement comme valeurs discrètes. Pourtant, la taille et le poids **d’un individu** peuvent prendre, si mesurés précisément, n’importe quelle valeur dans un intervalle (p. ex. : de 0 à 300 cm) et exprimer ainsi une continuité.

Comment, alors, être sûr·es de faire le bon choix ? Dans le doute, une bonne option est de se reposer sur la représentation graphique de la variable en ballottage :
- Un diagrammes en barres pour une quantitative discrète ;
- un histogramme pour une quantitative continue.

Dans notre petit jeu de données, nous avons catalogué la variable *année* comme une quantitative discrète. Représentons-la avec un diagramme en barres, puis un histogramme :

In [ ]:
figure, (ax_1, ax_2) = plt.subplots(ncols=2, figsize=(10,4))

sns.countplot(data=df, x="birth", ax=ax_1)
sns.histplot(data=df, x="birth", ax=ax_2)

ax_1.set(title="Un diagramme en barres", xlabel="Année", ylabel="Nombre")
ax_2.set(title="Un histogramme", xlabel="Année", ylabel="Nombre")

sns.despine()
plt.show()

Quand le premier graphique parle de lui-même, le second peine à convaincre.

## La délicate question du pré-traitement des données

Un algorithme de *machine learning* est grandement dépendant de la qualité des données sur lesquelles il est entraîné. Pour cette raison, la phase de pré-traitement (*pre-processing*) est cruciale. Il s’agira de ne laisser aucune donnée manquante dans le jeu de données et d’harmoniser les grandeurs des variables numériques.

### Distinguer les variables explicatives

Toutes les variables aléatoires d’une enquête ne servent pas forcément à expliquer un trait. Si l’ambition d’un algorithme est par exemple de prédire l’appartenance d’un manchot à telle ou telle espèce, son sexe ne sera pas un facteur déterminant, alors que la longueur de ses nageoires le serait probablement.

Commençons par charger le résultat d’une enquête sur la satisfaction des patients à l’hôpital :

In [ ]:
df = pd.read_csv("./data/satisfaction-hopital.csv", delimiter=";")

Afin de ne pas décider abritrairement des variables à sélectionner, on dresse une matrice de corrélation où le score révèle leur dépendance avec la variable cible. Sur un *data frame*, il suffit d’appeler la méthode `.corr()` :

In [ ]:
target = "recommander"
correlation_matrix = df.corr()

correlation_matrix[target]

Par défaut, le score calculé est le coefficient de corrélation de Pearson (ou *r* de Pearson), qui rend compte de la dépendance entre deux variables. On parle bien de dépendance et non pas de lien de causalité. Échelonné dans un intervalle situé entre $\mathopen{[}-1;1\mathclose{]}$, il indique la sensibilité d’une variable de la paire à la variance de l’autre. Un score de $1$ montre ainsi une corrélation linéaire positive, un score de $-1$ une corrélation linéaire négative, et un score de $0$ qu’il n’existe aucune corrélation entre les variables.

À partir de la matrice de corrélation, il est facile d’obtenir une carte thermique, parfois plus facile à lire :

In [ ]:
_ = sns.heatmap(correlation_matrix)

Dans l’exemple de l’enquête sur la satisfaction à l’hôpital, on remarque que la variable *recommander* est quelque peu dépendante du score attribué par le patient à la relation avec le personnel soignant et à la qualité de l’information qu’il a reçue. Comme les coefficients de corrélation sont assez faibles, on pourrait tout aussi bien retenir comme variables explicatives la perception de l’amélioration du moral et de l’état de santé du patient :

In [ ]:
features = ["score.relation", "score.information", "amelioration.sante", "amelioration.moral"]

Il ne reste plus qu’à reconstituer le jeu de données sur lequel nous allons entraîner l’algorithme d’apprentissage :

In [ ]:
data = pd.DataFrame(df[features + [target]])

### La chasse aux données manquantes…

Comment repérer les données manquantes dans un *dataset* et, surtout, comment les gérer ? Pour une seule variable sans valeur, faut-il supprimer toute l’observation ? Et s’il est question de la remplacer, quelle valeur choisir ?

La méthode `.info()` d’un *data frame* nous donne un aperçu de l’étendue des données manquantes :

In [ ]:
data.info()

Il en ressort que les 534 observations ne sont pas toutes complètes, loin s’en faut.

#### Supprimer les observations avec données manquantes

La première stratégie consiste à supprimer les observations concernées. Les *data frames* disposent d’une méthode `.dropna()` :

In [ ]:
data_copy = data.dropna()

data_copy.info()

L’opération a détruit pratiquement 50 % de l’échantillon, un rapport qui nous semble trop conséquent pour ce jeu de données. Une meilleure stratégie consisterait à combler les manques.

Pour autant, il est primordial que la série contenant la variable cible soit complète. Aussi, supprimons les observations où la variable *recommander* n’a pas été renseignée :

In [ ]:
# select rows where target is not null
data = data[data[target].notna()]
data.info()

Et, par commodité, réindexons le *data frame* avec la méthode `.reset_index()` :

In [ ]:
data = data.reset_index(drop=True)

#### Remplacer par une valeur

Plusieurs options se présentent : remplacer par des zéros, par une valeur fixe, par la moyenne, par la médiane ou encore par la valeur la plus représentée. Chacune de ces options a ses avantages et ses inconvénients.

*Scikit-Learn* dispose d’une classe `SimpleImputer` pour réaliser n’importe laquelle de ces options. Elle prend un paramètre `strategy`, dont les valeurs sont à choisir parmi : `mean` (option par défaut), `median`, `most_frequent`, `constant`. Si la stratégie `constant` est sélectionnée, il faut indiquer la valeur dans un paramètre `fill_value`.

Remplaçons dans un premier temps les valeurs manquantes par une valeur fixe, le zéro :

In [ ]:
from sklearn.impute import SimpleImputer

# copy of the data
data_copy = pd.DataFrame.copy(data)
# new instance
imputer = SimpleImputer(strategy="constant", fill_value=0)
# fit the imputer to data
imputer.fit(data_copy)
# create a matrice
X = imputer.transform(data_copy)

Dans les données avant traitement, la 7e observation contient deux données manquantes :

In [ ]:
data.iloc[6,:]

Après traitement, elles ont bien été fixées à 0 :

In [ ]:
# missing values for the 7th sample are now fixed to 0
X[6]

Cette méthode étant trop destructive, remplaçons plutôt par la valeur médiane :

In [ ]:
# copy
data_copy = pd.DataFrame.copy(data)
# an imputer with median strategy
imputer = SimpleImputer(strategy="median")
# shortcut for fit then transform
X = imputer.fit_transform(data_copy)

# values for the 7th sample
X[6]

D’autres imputeurs existent :
- `KNNImputer`, pour estimer les valeurs manquantes en fonction des valeurs voisines ;
- `IterativeImputer`, qui estime les valeurs à imputer pour chaque caractéristique à partir de toutes les autres.

### … et aux données aberrantes

Les mêmes stratégies peuvent s’appliquer aux données aberrantes, les valeurs extrêmes pouvant affecter négativement certaines mesures. Il s’agit parfois d’un zéro surnuméraire ou du déplacement de la virgule dans la notation décimale d’une quantité. La moyenne arithmétique est par exemple très sensible à ces erreurs. Et il en va de même des algorithmes d’apprentissage automatique.

### Séparer les données en jeux d’entraînement et de test

Il serait tout à fait possible de programmer un système intelligent sur l’ensemble des données à disposition, mais il ne permettrait alors pas de rendre compte de sa performance sur des données qui lui sont inconnues. Pour cette raison, il est d’usage de constituer un jeu pour l’entraîner et un autre pour le mettre à l’épreuve.

*Scikit-Learn* propose une fonction `train_test_split()` pour faciliter leur mise en place. Non seulement elle séparera les variables explicatives et la variable cible en deux jeux, mais elle les aura au préalable mélangées afin d’éviter tout biais dû au tri des observations :

In [ ]:
from sklearn.model_selection import train_test_split

# two data frames for: i) the target; ii) the features
y = data[target]
X = data.drop(columns=target)

# two sets for both data frames
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

Le paramètre `test_size` permet d’indiquer la proportion à utiliser. L’usage recommande un partage 80/20, 80 % pour le jeu d’entraînement et 20 % pour le jeu de test, mais il est à ajuster en fonction de la quantité de données à disposition.

Quant au paramètre `random_state`, il permet de contrôler le générateur de nombres aléatoires utilisé. Lui transmettre un entier naturel permet d’assurer la reproductibilité de la transformation. La valeur 42 est souvent renseignée parce qu’il s’agit de la réponse à la [question ultime sur la vie, l’Univers et tout le reste](https://en.wikipedia.org/wiki/Phrases_from_The_Hitchhiker%27s_Guide_to_the_Galaxy#The_Answer_to_the_Ultimate_Question_of_Life,_the_Universe,_and_Everything_is_42).

### Mise à l’échelle

Les données d’une observation font rarement toutes référence à une échelle commune. L’âge d’un individu sera compris entre 0 et 100, sa taille entre 0 et 200, son score de satisfaction entre 0 et 10, la numération de ses globules rouges entre 3 000 000 et 6 000 000 etc. Il faut savoir que les algorithmes d’apprentissage sont sensibles à la différence entre les grandeurs et fourniront des prédictions de mauvaise qualité si certaines variables sont réparties dans un espace bien plus vaste que les autres.

La mise à l’échelle consiste alors à réduire leur variance ou leur valeur absolue. Plusieurs méthodes existent et, parmi les plus utilisées, citons la **standardisation** et la **normalisation**.

Avant toutes choses, récupérons une variable descriptive de la satisfaction des patients à l’hôpital, le score de relation avec le personnel soignant et comblons manuellement les données manquantes :

In [ ]:
score_relation_median = data["score.relation"].median()
score_relation = data["score.relation"].fillna(score_relation_median)

#### La standardisation

La standardisation (*Z score normalization*) consiste à centrer la variable autour de 0 de telle manière que son écart-type soit égal à 1. La formule donne avec $\mu$ pour la moyenne et $\sigma$ pour l’écart-type :

$$f(x) = \frac{x − \mu}{\sigma}$$

Avant de centrer-réduire la variable *score_relation*, l’affichage de sa moyenne et de son écart-type donne 35,33 pour la première et 4,25 pour la seconde.

In [ ]:
print(
    f"Mean value: { score_relation.mean().round(2) }",
    f"Standard deviation: { score_relation.std().round(2) }",
    sep="\n"
)

Concrètement, l’opération de standardisation va d’abord soustraire la moyenne puis diviser ensuite le résultat par l’écart-type. Si l’on effectue ce calcul à la main, on obtient bien une moyenne à 0 et un écart-type de 1 :

In [ ]:
score_relation_scaled = [
    (n - score_relation.mean()) / score_relation.std()
    for n in score_relation
]
score_relation_scaled = pd.Series( (value for value in score_relation_scaled) )

print(
    f"Mean value: { score_relation_scaled.mean().round(2) }",
    f"Standard deviation: { score_relation_scaled.std().round(2) }",
    sep="\n"
)

Il existe heureusement une classe `StandardScaler` dans *Scikit-Learn* pour effectuer l’opération plus simplement :

In [ ]:
from sklearn.preprocessing import StandardScaler

# standard scaler
scaler = StandardScaler()

# reshape Serie to match 2d array
score_relation_scaled = scaler.fit_transform(score_relation.values.reshape(-1, 1))

print(
    f"Mean value: { score_relation_scaled.mean().round(2) }",
    f"Standard deviation: { score_relation_scaled.std().round(2) }",
    sep="\n"
)

#### La normalisation

Plus simple à appréhender, la normalisation Min-Max (*Min-Max normalization*) est une méthode qui va soustraire à chaque valeur la minimale puis la diviser ensuite par l’écart maximal de la série. Comme la formule est basée sur les extrêmes, elle est particulièrement sensible aux données aberrantes.

$$f(x) = \frac{x − min(x)}{max(x) − min(x)}$$

Le résultat n’est plus une variable centrée réduite, mais une variable dont les valeurs seront contenues dans un intervalle $[0, 1]$.

En reprenant l’exemple précédent sur le score de relation, nous appliquons cette fois-ci une classe `MinMaxScaler` :

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
score_relation_scaled = scaler.fit_transform(score_relation.values.reshape(-1, 1))

score_relation_scaled[:5]

## La mise en place d’un pipeline

Les pipelines en *machine learning* ne servent pas à convoyer des fluides, comme par exemple du gaz naturel ou du pétrole brut, mais plutôt des données. Comme les étapes de transformation sont parfois nombreuses et qu’il est impératif de les exécuter dans le bon ordre (remplacer les données manquantes avant de mettre les séries à l’échelle, etc.), *Scikit-Learn* fournit une fonction `make_pipeline()` pour garantir la cohérence entre elles :

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler(),
    LinearRegression()
)

Un pipeline est constitué de plusieurs estimateurs. Tous les estimateurs, à l’exception du dernier, doivent être des transformateurs.

Comme la dernière étape est assurée par un prédicteur, il suffit maintenant d’appeler la méthode `.fit()` pour ajuster le modèle sur les données d’entraînement :

In [ ]:
_ = model.fit(X_train, y_train)

Un autre avantage non négligeable des pipelines est qu’ils sont nettement plus rapides que si l’on réalisait les étapes une à une.

## Effectuer des prédictions

La méthode `.predict()` permet d’effectuer des prédictions. En lui envoyant les données du jeu de test (`X_test`), cela permet de comparer avec les résultats obtenus pour la variable cible (`y_test`) :

In [ ]:
# make some predictions
model.predict(X_test)

# evaluate (R2 score, mainly in [0, 1])
model.score(X_test, y_test)

Le coefficient de détermination mesure ici la qualité de la prédiction du modèle de régression linéaire. D’ordinaire situé dans l’intervalle $[0,1]$, un score de 0,43 montre la faiblesse de notre modèle, mais ne serait-il pas plutôt ici révélateur d’une erreur de méthode ? Quelle serait-elle selon vous ?

### Prédire avec des données inédites

Afin d’obtenir des prédictions à partir de données originales, la seule contrainte est qu’elles respectent le format des données sur lesquelles la machine a été entraînée. Prenons l’exemple de trois nouveaux patients qui attribuent les scores suivants aux quatre variables explicatives retenues par notre modèle :

In [ ]:
# unseen data…
new_data = {
    "score.relation": [32, 36, 31],
    "score.information": [37, 25, 30],
    "amelioration.sante": [3, 2, 1],
    "amelioration.moral": [2, 3, 3]
}

# … as a data frame
new_data = pd.DataFrame(new_data)

# get predictions
model.predict(new_data)

## Récapitulatif

Si nous reprenons les différentes étapes de la programmation de notre petite intelligence artificielle :

### 1e étape : extraire les variables cible et explicatives

L’analyse du jeu de données permet de déterminer quelles sont les variables explicatives à retenir pour programmer au mieux l’IA :

In [ ]:
df = pd.read_csv("./data/satisfaction-hopital.csv", delimiter=";")

# uncomment to display the correlation between all variables
#_ = sns.heatmap(df.corr())

target = "recommander"
features = ["score.relation", "score.information", "amelioration.sante", "amelioration.moral"]

data = pd.DataFrame(df[features + [target]])

### 2e étape : établir le jeu d’entraînement et le jeu de test

Dans un premier temps, seules sont retenues les observations pour lesquelles la donnée de la variable cible est connue. Toutes les autres sont supprimées. Une fonction de *Scikit-Learn* se charge ensuite de la répartition des données entre tous les jeux nécessaires (deux jeux pour d’un côté la variable cible ; deux autres pour les caractéristiques retenues) :

In [ ]:
# drop rows where target is missing
data.dropna(subset=target, inplace=True)

# two data frames for: i) the target; ii) the features
y = data[target]
X = data[features]

# two sets for both data frames
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### 3e étape : définir les couches du modèle

Dans un modèle, on retrouve une succession de programmes, appelés **estimateurs**, où chaque couche reçoit les données transformées par la précédente. Ainsi, tous les estimateurs à l’exception du dernier sont des **transformateurs**. À la dernière couche se tient le **prédicteur**, l’algorithme avec lequel les données seront entraînées afin d’effectuer des prédictions sur de nouvelles informations :

In [ ]:
model = make_pipeline(
    # fill the NA values
    SimpleImputer(strategy="median"),
    # scale data
    StandardScaler(),
    # a linear regression
    LinearRegression()
)

### 4e étape : entraîner le modèle et l’évaluer

Les méthodes `.fit()` et `.score()` du modèle servent à réaliser les deux actions. Il convient de transmettre à la première les jeux d’entraînement constitués à partir des caractéristiques (`X_train`) et à partir de la variable cible (`y_train`) ; et à la seconde les jeux de test correspondants (`X_test` et `y_test`) :

In [ ]:
# train…
model.fit(X_train, y_train)

# … then evaluate!
model.score(X_test, y_test)